# Lab — Altair

You have probably heard of tools such as Tableau to create visualisation dashboards.  Tableau is a great tool, but it has two primary limitations: 1) while quite powerful, sometimes you need to do more. 2) A lot of statistical data work is done programmatically rather than through a drag-n-drop interface.

In this lab, we will work with a dataset of Places-in-France using [Altair][], a Python library for creating statistical visualizations.

[Altair]: https://altair-viz.github.io

The goals of this lab are:

* Get a better understanding of grammar of statistical visualisations: spaces, mappings, marks, and encodings,
* Give you a sense of a programmatic way of creating an interactive visualisation,
* Understand the declarative way of thinking used by Altair, Vega-Lite, and D3.

In the first half of this notebook, most of the solutions are directly provided.  You can just read paragraph by paragraph and follow along in the code.  I encourage you to play with the code and try things as you move along.

Towards the end, there will be some blanks in the code that you will need to fill in.

Finally, there are some open exercises for you to complete the lab.

With that in mind, let's get started.

## Loading data

You should already have our [Places-in-France dataset][data] with this notebook.  If not, download it and place it in the same folder as this notebook.

We'll now need to import it into our project.  Altair can manage multiple data formats.  In our case, we'll use [Pandas][] to import our places-in-France data from CSV format.  (In addition to Pandas dataframes, Altair can also directly handle CSV, TSV, and JSON.)

In our case, we're going to just use the name of our CSV file.  You could also use a URL or a full path.

[Pandas]: https://pandas.pydata.org
[data]: https://perso.telecom-paristech.fr/eagan/class/igr204/data/france.csv

In [1]:
# Import the Altair library
# by convention, we use alt to save typing, but you can just 'import altair' if you wish
import altair as alt 
alt.data_transformers.enable('json') # work-around to let Altair handle larger data sets
# alt.renderers.enable('default') # In some old versions of Jupyter, you may need to enable this.

import pandas as pd


# Reference to data
france = pd.read_csv('france.csv')

# We could have also used a URL to a CSV file directly, but working with Pandas is a little easier.
# france = 'france.csv'

## Making a basic chart

Recall that our data set has columns `Postal Code`, `x`, `y`, `inseecode`, `place`, `population`, and `density`.  Let's make a map that plots the `x` vs `y` columns as points. 

In the code below, we'll make a `Chart` from the `france` data using `point` marks.  We'll bind (or _encode_ the chart's `x=` axis to the `x` attribute of our data set (and similarly for `y`).  In both cases, we tell Altair that the data are `Q`uantitative with `:Q`.  (There's also `:N` for nominal/categorical data, `:O` for ordinal, and `:T` for temporal data.)

In [2]:
map = alt.Chart(france).mark_point().encode(
    x='x:Q',
    y='y:Q'
)

map

alt.Chart(...)

That looks nice, but notice that Altair shows the origin (0, 0) by default.  That's often what you want, but not in this case.  Let's tell Altair to hide the axes and the origin.

In [3]:
## Hmm, looks nice, but let's change the default scales.
## Notice the change from shorthand notation.

map = alt.Chart(france).mark_point().encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False))
)

map

alt.Chart(...)

In the first map, we had used Altair's shorthand syntax to specify the data bindings.  That works for the common case, but here we need more control, so we have to use an explicit object for the `x` and `y` dimensions: `alt.X(...)` and `alt.Y(...)`.  We also need to create a `Scale` to set its `zero=False`.

This new chart looks much better, but we really can't see much detail here.  Let's reduce the default mark `size` to `1`.  We're also going to adjust the chart `properties` to set the width and height to something bigger.

In [4]:
## Much better, but we really can't see much here. 
## Let's make the point sizes smaller and make the canvas bigger

map = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False))
).properties(
    width=800,
    height=800
)

map

alt.Chart(...)

In Altair, a chart is made up of three primary things:

* Marks — the kinds of shapes to draw
* Encodings — a mapping from data to attributes of the marks
* Properties — meta-data about the chart

Now that we've seen each of them, let's talk a little more about each.

### Marks

_Marks_ are the kinds of shapes that we want to draw.  Here is a summary of the kinds of marks in Altair, taken from the [Altair documentation](https://altair-viz.github.io/user_guide/marks.html).

| Mark Name  | Method            |   Description                                    |
| ---------- | ----------------- |------------------------------------------------- |
| area       | `mark_area()`     | A filled area plot.                              |
| bar        | `mark_bar()`      | A bar plot.                                      |
| circle     | `mark_circle()`   | A scatter plot with filled circles.              |
| geoshape   | `mark_geoshape()` | A geographic shape                               |
| line       | `mark_line()`     | A line plot.                                     |
| point      | `mark_point()`    | A scatter plot with configurable point shapes.   |
| rect       | `mark_rect()`     | A filled rectangle, used for heatmaps            |
| rule       | `mark_rule()`     | A vertical or horizontal line spanning the axis. |
| square     | `mark_square()`   | A scatter plot with filled squares.              |
| text       | `mark_text()`     | A scatter plot with points represented by text.  |
| tick       | `mark_tick()`     | A vertical or horizontal tick mark.              |

The basic idea is that each data point will get mapped into one of these types of marks.

### Encodings

Encodings determine the binding between the data point and the mark.  In our example, we've been encoding the `x` and `y` data columns to each mark's `x`- and `y`-position.  As you can see in the [Altair documentation](https://altair-viz.github.io/user_guide/encoding.html#encoding-channels), there are positional encodings, mark property encodings (as we'll see in the next step), and interaction encodings (as we'll see later for with tooltips).

Let's go ahead and bind the size of the places to their population in our data set.  In other words, we're going to _encode_ the `population` data attribute by the _size_ of each _mark_.

#### Original method

In [5]:
map = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    
    # NEW: bind population to size
    # size='population:Q',
    
    # Or, try this alternate binding that clamps the range to [1, 400] instead of the default, [0, 400].
    size=alt.Size('population:Q', scale=alt.Scale(range=[1, 400])),
    
).properties(
    width=800,
    height=800
)

map

alt.Chart(...)

That looks great!  We can already start to see some interesting emergent features from our data, such as the effect of rivers, mountains, and other influences.

Notice the alternative solution in the above code.  Go ahead and try the other solution.  Do you see a difference in the output?  What do you think is going on?

#### Try testing the Alternative solution

In [6]:
map = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    
    # NEW: bind population to size
    size='population:Q',
    
    # Or, try this alternate binding that clamps the range to [1, 400] instead of the default, [0, 400].
    # size=alt.Size('population:Q', scale=alt.Scale(range=[1, 400])),
    
).properties(
    width=800,
    height=800
)

map

alt.Chart(...)

#### Q1: What changes between the two solutions?

With the first solution we specified the scale for the size of each ``mark_point`` namely from 1 to 400 (i.e. ``scale=alt.Scale(range=[1, 400]``)).

With the alternative solution we did not specify the scale for the size of each ``mark_point``. So in this case the default settings are applied. That is why, with the alternative solution, we notice that the map of France is less overloaded compared to the map of France generated by the original solution. Indeed, this seems to be due to the fact that the scale of the ``mark_point`` size starts at 0. Thus, cities with a very small population are not displayed on the map. 

To put in a nutshell, by varying the scale interval we notice that the wider the chosen interval the more contrast we can observe between cities with a high population and cities with a very low population. This can be explained by the fact that the difference between the size of the smallest and the largest point increases proportionally to the chosen range.

So far, we have bindings for `x`, `y`, and `population`.  In the code below, go ahead and bind <!-- comment --> `density` to each place's color.  _Hint_: you declare a color encoding with `color=`.

<!-- 
    Simple solution:  color='density:Q'

    Complex solution:
        color=alt.Color('density:Q', type='quantitative', scale=alt.Scale(type='sqrt', range=['lightblue', 'red'])),

-->

#### Original solution

In [10]:
## How would we adapt this to encode density by color?
## Hint: you declare a color encoding with color=

map = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',

    # >>> your code here <<<
    # color="density:Q"
    
    # Alternative solution
    color=alt.Color("density:Q", scale = alt.Scale(scheme='tealblues', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map

alt.Chart(...)

#### Try alternative solution

In [11]:
## How would we adapt this to encode density by color?
## Hint: you declare a color encoding with color=

map = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',

    # >>> your code here <<<
    color="density:Q"
    
    # Alternative solution
    #color=alt.Color("density:Q", scale = alt.Scale(scheme='tealblues', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map

alt.Chart(...)

### Comments

We can draw the same conclusions as previously mentioned.

By varying the scale interval we notice that the wider the chosen interval the more contrast we can observe between cities with a high density population and cities with a very low density population. This can be explained by the fact that the difference between the size of the smallest and the largest point increases proportionally to the chosen range.

Moreover, by specifying the interval for the color scale, I find that with the original method one can more easily notice the difference between the areas where the population density is low and high. For example on the visualization generated by the original method we can see at a glance that the strong areas where the population density is high are located around the largest cities of France namely Paris first and then Lyon, Marseille, Lille,...

## Altair

So far, we've just gotten started with Altair.  Before we dig a little deeper, let's take a closer look at the shorthand notation and the classes used in Altair.

By default, Altair tries to use reasonable defaults.  If you're just exploring data, they're often good enough.  That will let you create basic charts just by binding, say, `x` and/or `y` to some column in your data table, e.g. `x='population:Q', y='density:Q'`.

But if we need to override the defaults, we need to use a more complicated syntax.  Each of the encodings defines a configurable class.  Instead of using a shorthand string, as we did above, we can pass in an instance of that class.  That's what we did in the first step when we switched from using

    x='x:Q',
    
to

    x=alt.X('x:Q', axis=None)

How did we know we could control the axis through the `axis=` keyword?  The [Altair documentation](https://altair-viz.github.io/user_guide/encoding.html#encoding-channels) lists the different encoding classes available.  A click on the `X` class on that page will take you to the documentation that enumerates the different parameters you can override from their default values.  Notice that the third attribute down on that page shows:

    axis:anyOf(Axis, None)
    
It's a bit tricky to read, but it says that we can either pass in an instance of the Altair `Axis` class or the value `None`—in which case, the documentation explains, that axis will be removed from the chart.


## Compared to other approaches

You may have heard of or even used other libraries for creating statistical visualisations, including matplotlib, ggplot, Seaborn, and others.  Each of these libraries has a healthy community behind them and are perfectly reasonable choices to use, and most of them will use similar concepts to what we are seeing here with Altair.

In this class, however, we use Altair for several reasons:

* It has a nice mapping of Bertin's and Wilkinson's concepts for statistical visualisation (that we saw in class earlier today!) baked into their APIs,
* It uses a declarative rather than procedural design approach,
* You can easily export your visualisations to PNG, SVG, or even interactive web pages, and
* It generates Vega-Lite descriptions that are then rendered using D3, which we'll see later on.

That second point probably needs a little more explanation.  In a procedural syntax, you could imagine saying something like:

    for each data point:
        draw a circle of size datum.population at datum.x, datum.y

in our declarative approach, we'd say instead:

    make a chart for data where:
        position is (datum.x, datum.y) and size is datum.population

At first glance, those might look awfully similar, but the key difference is that, in the first case, we say _how_ to do what we want, whereas in the second case, we simply say _what_ we want.  Altair takes care of the rest.

Altair, D3, and Vega all use this second, declarative approach.  It takes some getting used to, but it is part of what makes these libraries compelling.

## Tooltips

So far, our visualisations are not particularly interactive.  Let's go ahead and change that.

The barest, most minimal interaction we can add is to show a tooltip when we hover over a place.  In Altair, we can use the [Tooltip](https://altair-viz.github.io/user_guide/generated/channels/altair.Tooltip.html) encoding.

In [12]:
# Let's add tooltips

map = map.encode(
    tooltip=['place:N', 'population:Q', 'density:Q']
)

map

alt.Chart(...)

## Multiple views

It would be nice to see a different representation of the distribution of population and densities in France.  Let's go ahead and add in a “histogram” of population and density….

A histogram is just a bar chart of the different bins of values, so let's create a chart using _bars_ for the _marks_.  We want to show the population on the _x_ axis and the number of people in that bin on the _y_ axis:

In [13]:
population = alt.Chart(france, width=800, height=100).mark_bar().encode(
    x='population:Q',
    y='count(population):Q',
)

population

alt.Chart(...)

That's close, but we're not really binning our data.  Let's go ahead and create slices (or bins) of the data on the *x*
axis.  (In French, we'd call these _tranches_.)

In [14]:
population = alt.Chart(france, width=800, height=100).mark_bar().encode(
    x=alt.X('population:Q', bin=True),
    y='count(population):Q',
)

population

alt.Chart(...)

That's better, but the default bins are a bit too big.  Let's override the number of bins.  Also, let's show the number of people in each bin instead of the number of places by replacing the `count()` with the `sum()`.

In [15]:
population = alt.Chart(france, width=800, height=100).mark_bar().encode(
    x=alt.X('population:Q', bin=alt.Bin(maxbins=60)),
    y='sum(population):Q',
)

population

alt.Chart(...)

### Density histogram

How would you make a histogram of the densities?  Try changing the code below do make it into a _density_ histogram instead of showing the population.

In [18]:
density = alt.Chart(france, width=800, height=100).mark_bar().encode(
    x=alt.X('density:Q', bin=alt.Bin(maxbins=60)),
    y='sum(density):Q',
)

density

alt.Chart(...)

# Making a multi-view visualisation (e.g., “dashboard”)

We now have all of the pieces we need to make a multi-view visualisation (what we called a “dashboard” in Tableau).

In Altair, you can combine multiple charts manually using the `&` and `|` operators to combine them _vertically_ or _horizontally_, respectively.  (There's also `+` to overlay them on top of each other, but we don't need that here.)

In [19]:
population & density & map

alt.VConcatChart(...)

## Other interactions

Beyond tooltips, we want to be able to _select_ data points, _filter_ out data, and to be able to _link views_ so that, say, a selection in one might filter what is shown in another (or alter its presentation in some other way).

Let's start with selections.  There are three kinds of selections:

* intervals — a range of values
* single - a single value
* multi - multiple values

Intervals let the user “brush” over a range of values to select them.  Single selection select a single data point, while multi selections let the user combine single selections (by default with the shift key).  [For more info, see the Altair selections guide](https://altair-viz.github.io/user_guide/interactions.html)

Here, let's add a rectangular selection brush to our map.

In [20]:
brush = alt.selection_interval()
map.add_selection(brush)

alt.Chart(...)

Now try it out.  Try dragging a box around a part of the map.

---

Huh, that sort of works.  We see a selection appear, but it doesn't _do_ anything with it.  Let's fix that.

In [21]:
# Let's gray out unselected places

map = alt.Chart(france, width=800, height=800).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    color=alt.condition(brush, 'density:Q', alt.value('lightgrey')),
).add_selection(brush)

map

alt.Chart(...)

Well, that seems to work.  It's a little slow (because we have a _lot_ of _large_ views open in this notebook), but moreover, it doesn't really _do_ much useful.

Let's go ahead and link our selection to the histogram.  Notice that all we need to do to filter the histogram based on the selection is to add a `.transform_filter()` to the histogram that uses the map's selection `brush`.  (See the last line of the histogram.)

In [22]:
brush = alt.selection_interval()

map = alt.Chart(france, width=800, height=800).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    color='density:Q',
).add_selection(brush)

population = alt.Chart(france, width=800, height=100).mark_bar().encode(
    x=alt.X('population:Q', bin=alt.Bin(maxbins=60)),
    y='sum(population):Q',
).transform_filter(brush)

population & map

alt.VConcatChart(...)

That's great, but it would be nice if we could link the views both ways: a selection in the map updates the histogram or a selection in the histogram updates the map.

To do that, we'll create two selections:

In [23]:
brush = alt.selection_interval()
pop_selection = alt.selection_interval(encodings=['x'])

Notice that the new selection uses an optional `encodings` parameter to indicate that it only selects data items along the `x` encoding from the chart (which we have mapped to the `x` data attribute).

Now update the code below to link the two charts using the new brush. (**The code below is incomplete.  It will show an error until you have linked things in correctly.**)

In [26]:
map = alt.Chart(france, width=800, height=800).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    color=alt.condition(pop_selection, 'density:Q', alt.value('lightgrey')),
).add_selection(brush)

# add the code : add._selection(pop_selection)
population = alt.Chart(france, width=800, height=100).mark_bar().encode(
    x=alt.X('population:Q', bin=alt.Bin(maxbins=60)),
    y='sum(population):Q',
).add_selection(pop_selection).transform_filter(brush)

population & map

alt.VConcatChart(...)

## Exploration

* Add linked selection to density “histogram”
* Try different encodings: explore [alt.Scale][], [alt.Color][], etc.
* Try different marks: mark_rect, mark_tick, mark_circle, …. [Marks documentation][]
* Try making a heatmap of places in France.

[alt.Scale]: https://altair-viz.github.io/user_guide/generated/core/altair.Scale.html#altair.Scale
[alt.Color]: https://altair-viz.github.io/user_guide/generated/channels/altair.Color.html#altair.Color
[Marks documentation]: https://altair-viz.github.io/user_guide/marks.html

### Add linked selection to density histogram

I linked selection to density and population historgram

In [31]:
brush = alt.selection_interval()
density_selection = alt.selection_interval(encodings=['x'])

map = alt.Chart(france, width=800, height=800).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    color=alt.condition(density_selection, 'density:Q', alt.value('lightgrey')),
).add_selection(brush)

# add the code : add._selection(pop_selection)
population = alt.Chart(france, width=800, height=100).mark_bar().encode(
    x=alt.X('population:Q', bin=alt.Bin(maxbins=60)),
    y='sum(population):Q',
).add_selection(density_selection).transform_filter(brush)

density = alt.Chart(france, width=800, height=100).mark_bar().encode(
    x=alt.X('density:Q', bin=alt.Bin(maxbins=60)),
    y='sum(density):Q',
).add_selection(density_selection).transform_filter(brush)

population & density & map 

alt.VConcatChart(...)

### Try different encodings: explore alt.Scale

Test different categories of scale types: [ “linear”, “pow”, “sqrt”, “symlog”, “log”, “time”, “utc”]

#### ``alt.Scale(type = "sqrt")`` and ``alt.Scale(scheme = "tealblues")``

In [40]:
map_sqrt = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different categories of scale types: [ “linear”, “pow”, “sqrt”, “symlog”, “log”, “time”, “utc”]
    color=alt.Color("density:Q", scale = alt.Scale(scheme='tealblues', type = 'sqrt', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map_sqrt

alt.Chart(...)

#### ``alt.Scale(type = "pow")`` and ``alt.Scale(scheme = "reds")``

In [38]:
map_pow = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different categories of scale types: [ “linear”, “pow”, “sqrt”, “symlog”, “log”, “time”, “utc”]
    color=alt.Color("density:Q", scale = alt.Scale(scheme='reds', type = 'pow', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map_pow

alt.Chart(...)

#### ``alt.Scale(type = "time")`` and ``alt.Scale(scheme = "reds")``

In [39]:
map_time = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different categories of scale types: [ “linear”, “pow”, “sqrt”, “symlog”, “log”, “time”, “utc”]
    color=alt.Color("density:Q", scale = alt.Scale(scheme='browns', type = 'time', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map_time

alt.Chart(...)

#### Try different encodings: explore alt.Color

Test different type of aggregation : [ “mean”, “median”, “max”, “min”, “count”]

#### ``alt.Color(aggregate = "mean")``

In [45]:
map_mean = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different 
    color=alt.Color("density:Q", aggregate = "mean", scale = alt.Scale(scheme='browns', type = 'linear', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map_mean

alt.Chart(...)

#### ``alt.Color(aggregate = "count")``

In [57]:
map_count = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different 
    color=alt.Color("density:Q", aggregate = "count", scale = alt.Scale(scheme='tealblues', type = 'sqrt', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map_count

alt.Chart(...)

#### ``alt.Color(aggregate = "min")``

In [55]:
map_min = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different 
    color=alt.Color("density:Q", aggregate = "min")

).properties(
    width=800,
    height=800
)

map_min

alt.Chart(...)

#### ``alt.Color(aggregate = "sum")``

In [60]:
map_sum = alt.Chart(france).mark_point(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different 
    color=alt.Color("density:Q", aggregate = "sum", scale = alt.Scale(scheme='warmgreys', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map_sum

alt.Chart(...)

### Try different marks: mark_rect, mark_tick, mark_circle, …. Marks documentation

#### ``mark_cicle(size=1)``

In [70]:
map_circle = alt.Chart(france).mark_circle(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different categories of scale types: [ “linear”, “pow”, “sqrt”, “symlog”, “log”, “time”, “utc”]
    color=alt.Color("density:Q", scale = alt.Scale(scheme='tealblues', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map_circle

alt.Chart(...)

#### ``mark_circle(opacity = 0.8)``

In [82]:
map_circle_triangle_down = alt.Chart(france).mark_circle(size=1, opacity = 0.8).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different categories of scale types: [ “linear”, “pow”, “sqrt”, “symlog”, “log”, “time”, “utc”]
    color=alt.Color("density:Q", scale = alt.Scale(scheme='reds'))

).properties(
    width=800,
    height=800
)

map_circle_triangle_down

alt.Chart(...)

#### ``mark_square(size=1)``

In [79]:
map_square = alt.Chart(france).mark_square(size=100, opacity=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different categories of scale types: [ “linear”, “pow”, “sqrt”, “symlog”, “log”, “time”, “utc”]
    color=alt.Color("density:Q", scale = alt.Scale(scheme='tealblues'))

).properties(
    width=800,
    height=800
)

map_square

alt.Chart(...)

#### ``mark_trail(size=1)``

In [74]:
map_square = alt.Chart(france).mark_trail(size=1).encode(
    x=alt.X('x:Q', axis=None),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False)),
    size='population:Q',
    
    # try different alt.Scale 
    # Test different categories of scale types: [ “linear”, “pow”, “sqrt”, “symlog”, “log”, “time”, “utc”]
    color=alt.Color("density:Q", scale = alt.Scale(scheme='tealblues', domain=[1,600]))

).properties(
    width=800,
    height=800
)

map_square

alt.Chart(...)

#### This type of visualization is a failure because one does not recognize the carte france anymore. Therefore this type of marker is not suitable for our data

## Exercises

* Try making a heatmap of places in France
    * _hint_: try setting color to `'count()'`

![heatmap-france.png](heatmap-france.png)

In [96]:
# Your code here

heatmap_population = alt.Chart(france).mark_square(size=20, opacity=1).encode(
    x=alt.X('x:Q', axis=None, bin=alt.Bin(maxbins=200)),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False), bin=alt.Bin(maxbins=200)),
    color=alt.Color('count(population):Q', scale=alt.Scale(scheme='greens'))
).properties(
    width=800,
    height=800
)

heatmap_population

alt.Chart(...)

* Or a heatmap of population densities in France
    * _hint_: The solution below binds color to `'mean(population):Q'` and uses a `Scale` with `scheme='oranges'`

![heatmap-densities](heatmap-density.png)

In [99]:
# Your code here

heatmap_density = alt.Chart(france).mark_square(size=20, opacity=1).encode(
    x=alt.X('x:Q', axis=None, bin=alt.Bin(maxbins=200)),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False), bin=alt.Bin(maxbins=200)),
    color=alt.Color('mean(density):Q', scale=alt.Scale(scheme='oranges'))
).properties(
    width=800,
    height=800
)

heatmap_density

alt.Chart(...)

* Or of both population and density:

![heatmap both](heatmap-density-population.png)

In [124]:
# Your code here


#### heatmap population ####
heatmap_population = alt.Chart(france).mark_square(size=30, opacity=0.8).encode(
    x=alt.X('x:Q', axis=None, bin=alt.Bin(maxbins=150)),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False), bin=alt.Bin(maxbins=150)),
    color=alt.Color('mean(population):Q', scale=alt.Scale(scheme='oranges'))
).properties(
    width=500,
    height=500,
    title = "Population Heatmap"
)


#### heatmap density ####

heatmap_density = alt.Chart(france).mark_square(size=30, opacity=1).encode(
    x=alt.X('x:Q', axis=None, bin=alt.Bin(maxbins=150)),
    y=alt.Y('y:Q', axis=None, scale=alt.Scale(zero=False), bin=alt.Bin(maxbins=150)),
    color=alt.Color('mean(density):Q', scale=alt.Scale(scheme='oranges'))
).properties(
    width=500,
    height=500,
    title = "Density popluation Heatmap"
)

heatmap_density | heatmap_population

alt.HConcatChart(...)